In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import matplotlib.colors as mcolors
import random
import math
import time
from sklearn.model_selection import RandomizedSearchCV, train_test_split
from sklearn.svm import SVR
from sklearn.metrics import mean_squared_error, mean_absolute_error 
import datetime
import operator
plt.style.use('seaborn')
%matplotlib inline

In [2]:
#import data
deaths_reported=pd.read_csv('deaths_global.csv')
confirmed_cases=pd.read_csv('confirmed_global.csv')
recovered_cases=pd.read_csv('recovered_global.csv')

In [3]:
#displayings heads of the data
confirmed_cases.head()


,Province/State,Country/Region,Lat,Long,1/22/2020,1/23/2020,1/24/2020,1/25/2020,1/26/2020,1/27/2020,...,05/05/2020,05/06/2020,05/07/2020,05/08/2020,05/09/2020,05/10/2020,05/11/2020,05/12/2020,5/13/2020,5/14/2020
0,NaN,Afghanistan,33.0000,65.0000,0,0,0,0,0,0,...,421,458,468,472,502,558,558,610,648,691
1,NaN,Albania,41.1533,20.1683,0,0,0,0,0,0,...,570,595,605,620,627,650,654,682,688,694
2,NaN,Algeria,28.0339,1.6596,0,0,0,0,0,0,...,2067,2197,2323,2467,2546,2678,2841,2998,3058,3158
3,NaN,Andorra,42.5063,1.5218,0,0,0,0,0,0,...,514,521,526,537,545,550,550,568,576,596
4,NaN,Angola,-11.2027,17.8739,0,0,0,0,0,0,...,11,11,11,11,13,13,13,13,14,14


In [4]:
deaths_reported.head()

,Province/State,Country/Region,Lat,Long,1/22/20,1/23/20,1/24/20,1/25/20,1/26/20,1/27/20,...,5/5/20,5/6/20,5/7/20,5/8/20,5/9/20,5/10/20,5/11/20,5/12/20,5/13/20,5/14/20
0,NaN,Afghanistan,33.0000,65.0000,0,0,0,0,0,0,...,95,104,106,109,115,120,122,127,132,136
1,NaN,Albania,41.1533,20.1683,0,0,0,0,0,0,...,31,31,31,31,31,31,31,31,31,31
2,NaN,Algeria,28.0339,1.6596,0,0,0,0,0,0,...,470,476,483,488,494,502,507,515,522,529
3,NaN,Andorra,42.5063,1.5218,0,0,0,0,0,0,...,46,46,47,47,48,48,48,48,49,49
4,NaN,Angola,-11.2027,17.8739,0,0,0,0,0,0,...,2,2,2,2,2,2,2,2,2,2


In [5]:
recovered_cases.head()

,Province/State,Country/Region,Lat,Long,1/22/20,1/23/20,1/24/20,1/25/20,1/26/20,1/27/20,...,5/5/20,5/6/20,5/7/20,5/8/20,5/9/20,5/10/20,5/11/20,5/12/20,5/13/20,5/14/20
0,NaN,Afghanistan,33.0000,65.0000,0,0,0,0,0,0,...,421,458,468,472,502,558,558,610,648,691
1,NaN,Albania,41.1533,20.1683,0,0,0,0,0,0,...,570,595,605,620,627,650,654,682,688,694
2,NaN,Algeria,28.0339,1.6596,0,0,0,0,0,0,...,2067,2197,2323,2467,2546,2678,2841,2998,3058,3158
3,NaN,Andorra,42.5063,1.5218,0,0,0,0,0,0,...,514,521,526,537,545,550,550,568,576,596
4,NaN,Angola,-11.2027,17.8739,0,0,0,0,0,0,...,11,11,11,11,13,13,13,13,14,14


In [6]:
#extracting all the columns using .key()
cols=confirmed_cases.keys()

In [7]:
#extracting only the dates column that have the confirmed cases, death reported and recovered cases
confirmed=confirmed_cases.loc[:,cols[4]:cols[-1]]

In [8]:
deaths=deaths_reported.loc[:,cols[4]:cols[-1]]

KeyError: '1/22/2020'

In [9]:
recoveries=recovered_cases.loc[:,cols[4]:cols[-1]]

KeyError: '1/22/2020'

In [95]:
# check the head of pandemic cases
confirmed.head()

,1/22/2020,1/23/2020,1/24/2020,1/25/2020,1/26/2020,1/27/2020,1/28/2020,1/29/2020,1/30/2020,1/31/2020,...,5/5/2020,5/6/2020,5/7/2020,5/8/2020,5/9/2020,5/10/2020,5/11/2020,5/12/2020,5/13/2020,5/14/2020
0,0,0,0,0,0,0,0,0,0,0,...,421,458,468,472,502,558,558,610,648,691
1,0,0,0,0,0,0,0,0,0,0,...,570,595,605,620,627,650,654,682,688,694
2,0,0,0,0,0,0,0,0,0,0,...,2067,2197,2323,2467,2546,2678,2841,2998,3058,3158
3,0,0,0,0,0,0,0,0,0,0,...,514,521,526,537,545,550,550,568,576,596
4,0,0,0,0,0,0,0,0,0,0,...,11,11,11,11,13,13,13,13,14,14


In [96]:
#finding the total confirmed cases, death cases and the recovered case and appending them on 4 empty lists
# calculate the total mortality rate which is death_sum/Confirmed cases
dates=confirmed.keys()
world_cases=[]
total_deaths=[]
mortality_rate=[]
total_recovered=[]

for i in dates:
    confirmed_sum=confirmed[i].sum()
    death_sum=deaths[i].sum()
    recovered_sum=recoveries[i].sum()
    world_cases.append(confirmed_sum)
    total_deaths.append(death_sum)
    mortality_rate.append(death_sum/confirmed_sum)
    total_recovered.append(recovered_sum)

In [97]:
#displaying the new variables
confirmed_sum

1587893

In [98]:
death_sum

302418

In [99]:
recovered_sum

1587893

In [100]:
world_cases

[28,
 30,
 36,
 39,
 52,
 61,
 107,
 126,
 143,
 222,
 284,
 472,
 623,
 852,
 1124,
 1487,
 2011,
 2616,
 3244,
 3946,
 4683,
 5150,
 6295,
 8058,
 9395,
 10865,
 12583,
 14352,
 16121,
 18177,
 18890,
 22886,
 23394,
 25227,
 27905,
 30384,
 33277,
 36711,
 39782,
 42716,
 45602,
 48228,
 51170,
 53796,
 55865,
 58358,
 60694,
 62494,
 64404,
 67003,
 68324,
 70251,
 72624,
 76034,
 78088,
 80840,
 83312,
 84975,
 87420,
 91692,
 97899,
 98351,
 108000,
 113787,
 122150,
 130915,
 139415,
 149082,
 164566,
 178034,
 193198,
 210269,
 225814,
 246152,
 260023,
 276534,
 300054,
 328703,
 353989,
 376104,
 402120,
 421722,
 448672,
 473980,
 510516,
 541592,
 567765,
 591719,
 623307,
 645308,
 679905,
 710046,
 738980,
 789585,
 817405,
 845985,
 873677,
 906955,
 948425,
 1013886,
 1052415,
 1093137,
 1125236,
 1162724,
 1198832,
 1245413,
 1284741,
 1322050,
 1375624,
 1408980,
 1456209,
 1493414,
 1548547,
 1587893]

In [101]:
mortality_rate

[0.6071428571428571,
 0.6,
 0.7222222222222222,
 1.0769230769230769,
 1.0769230769230769,
 1.3442622950819672,
 1.2242990654205608,
 1.0555555555555556,
 1.1958041958041958,
 0.9594594594594594,
 0.9119718309859155,
 0.7669491525423728,
 0.6837881219903692,
 0.5774647887323944,
 0.501779359430605,
 0.42636180228648285,
 0.3575335653903531,
 0.3081039755351682,
 0.2792848335388409,
 0.25671566142929547,
 0.23766816143497757,
 0.2170873786407767,
 0.21779189833200954,
 0.18900471581037478,
 0.17732836615220862,
 0.16290842153704557,
 0.14845426368910436,
 0.13984113712374582,
 0.13162955151665529,
 0.12361775870605711,
 0.11916357861302276,
 0.1074019050948178,
 0.10553988202103104,
 0.10421373924763151,
 0.0970435405841247,
 0.09116640337019484,
 0.08456291132013102,
 0.0782326823023072,
 0.07392790709366046,
 0.07013765333832755,
 0.0676505416429104,
 0.06552210334245666,
 0.06359194840726988,
 0.06221652167447394,
 0.061917121632506934,
 0.06096850474656431,
 0.06262563021056447,
 0.0

In [102]:
#convert all date and cases in the form of numpy array
days_since_1_22=np.array([i for i in range(len(dates))]).reshape(-1,1)
world_cases=np.array(world_cases).reshape(-1,1)
total_deaths=np.array(total_deaths).reshape(-1,1)
total_recovered=np.array(total_recovered).reshape(-1,1)

In [103]:
days_since_1_22

array([[  0],
       [  1],
       [  2],
       [  3],
       [  4],
       [  5],
       [  6],
       [  7],
       [  8],
       [  9],
       [ 10],
       [ 11],
       [ 12],
       [ 13],
       [ 14],
       [ 15],
       [ 16],
       [ 17],
       [ 18],
       [ 19],
       [ 20],
       [ 21],
       [ 22],
       [ 23],
       [ 24],
       [ 25],
       [ 26],
       [ 27],
       [ 28],
       [ 29],
       [ 30],
       [ 31],
       [ 32],
       [ 33],
       [ 34],
       [ 35],
       [ 36],
       [ 37],
       [ 38],
       [ 39],
       [ 40],
       [ 41],
       [ 42],
       [ 43],
       [ 44],
       [ 45],
       [ 46],
       [ 47],
       [ 48],
       [ 49],
       [ 50],
       [ 51],
       [ 52],
       [ 53],
       [ 54],
       [ 55],
       [ 56],
       [ 57],
       [ 58],
       [ 59],
       [ 60],
       [ 61],
       [ 62],
       [ 63],
       [ 64],
       [ 65],
       [ 66],
       [ 67],
       [ 68],
       [ 69],
       [ 70],
      

In [104]:
world_cases

array([[     28],
       [     30],
       [     36],
       [     39],
       [     52],
       [     61],
       [    107],
       [    126],
       [    143],
       [    222],
       [    284],
       [    472],
       [    623],
       [    852],
       [   1124],
       [   1487],
       [   2011],
       [   2616],
       [   3244],
       [   3946],
       [   4683],
       [   5150],
       [   6295],
       [   8058],
       [   9395],
       [  10865],
       [  12583],
       [  14352],
       [  16121],
       [  18177],
       [  18890],
       [  22886],
       [  23394],
       [  25227],
       [  27905],
       [  30384],
       [  33277],
       [  36711],
       [  39782],
       [  42716],
       [  45602],
       [  48228],
       [  51170],
       [  53796],
       [  55865],
       [  58358],
       [  60694],
       [  62494],
       [  64404],
       [  67003],
       [  68324],
       [  70251],
       [  72624],
       [  76034],
       [  78088],
       [  

In [105]:
total_deaths

array([[    17],
       [    18],
       [    26],
       [    42],
       [    56],
       [    82],
       [   131],
       [   133],
       [   171],
       [   213],
       [   259],
       [   362],
       [   426],
       [   492],
       [   564],
       [   634],
       [   719],
       [   806],
       [   906],
       [  1013],
       [  1113],
       [  1118],
       [  1371],
       [  1523],
       [  1666],
       [  1770],
       [  1868],
       [  2007],
       [  2122],
       [  2247],
       [  2251],
       [  2458],
       [  2469],
       [  2629],
       [  2708],
       [  2770],
       [  2814],
       [  2872],
       [  2941],
       [  2996],
       [  3085],
       [  3160],
       [  3254],
       [  3347],
       [  3459],
       [  3558],
       [  3801],
       [  3987],
       [  4263],
       [  4614],
       [  4722],
       [  5408],
       [  5830],
       [  6472],
       [  7150],
       [  7956],
       [  8823],
       [  9946],
       [ 11422

In [106]:
#forecasting the next ten days
days_in_future=10
future_forecast=np.array([i in range(len(dates)+days_in_future)]).reshape(-1,1)
adjusted_dates=future_forecast[:-10]

In [107]:
future_forecast

array([[False]])

In [108]:
#convert all the integers into datetime for better visualization
start='1/22/2020'
start_date=datetime.datetime.strptime(start, %m/%D/%Y)
future_forecast_dates=[]
for i in range(len(future_forecast)):
    future_forecast_dates.append((start_date+datetime.timedelta(days=i)).strptime('%m/%D/%Y'))

SyntaxError: invalid syntax (<ipython-input-108-2b9cf502f4e2>, line 3)

In [ ]:
#visualiizing data using the latest data
latest_confirmed=confirmed_cases[dates[-1]]
latest_deaths=deaths_reported[dates[-1]]
latest_recoveries=recovered_cases[dates[-1]]

In [ ]:
latest_confirmed

In [ ]:
latest_deaths

In [ ]:
latest_recoveries

In [ ]:
#list of unique countries
unique_countries=list(confirmed_cases['Country/Region'].unique())
unique_countries

In [ ]:
#total number of confirmed cases by each country
country_confirmed_cases=[]
no_cases=[]
for i in unique_countries:
    cases=latest_confirmed[confirmed_cases['Country/Region']==i].sum()
    if cases > 0:
        country_confirmed_cases.append(cases)
    else:
        no_cases.append(i)
for i in no_cases:
    unique_countries.remove(i)
unique_countries=[k for k,v in sorted(zip(unique_countries, country_confirmed_cases),key=operator.itemgetter(1),reverse=True)]
for i in range(len(unique_countries)):
    country_confirmed_cases[i]=latest_confirmed[confirmed_cases['Country/Region']==unique_countries[i]].sum()
    

In [ ]:
#number of cases per country/region.
print('confirmed cases by countries/Region:')
for i in range(len(unique_countries)):
     print(f'{unique_countries[i]}:{country_confirmed_cases[i]} cases')
        

In [ ]:
#list of unique provinces
unique_provinces=list(confirmed_cases['Province/State'].unique())
#those countries which are not province/state are considered s outliers
outliers=['United Kingdom','Denmark','France']
for i in outliers:
    unique_provinces.remove(i)


In [ ]:
#number of confirmed cases per state/province
province_confirmed_cases=[]
no_cases=[]
for i in unique_province:
    caes=latest_confirmed[confirmed_cases['Province/State']==i].sum()
    if cases>0:
        province_confirmed_cases.append(cases)
    else:
        no_cases.append(i)

for i in no_cases:
    unique_province.remove(i)


In [ ]:
#number of cases per Province/state
for i in range(len(unique_provinces)):
    print(f"{unique_provinces[i]}:{province_confirmed_cases[i]} cases")


In [ ]:
# dealing with nan values if any

nan_indices=[]
for i in range(len(unique_provinces)):
    if type(unique_provinces[i]) == float:
        nan_indices.append(i)

unique_provinces=list(unique_provinces)
provinces_confirmed_cases=list(province_confirmed_cases)

for i in nan_indices:
    unique_provinces.pop(i)
    provinces_confirmed_cases.pop(i)


In [ ]:
#VISUALIZING DATA
#bar gragh of all confirmed cass across different countries
plt.figure(figsize=(32, 32))
plt.barh(unique_countries, country_confirmed_cases)
plt.title('Number of Covid-19 Confirmed cases in countries')
plt.xlabel('Number of covid-19 Confirmed Cases')
plt.show()

In [ ]:
#show 10 countries with the most confirmed cases the rest of the countries are grouped into category called others
visual_unique_countries=[]
visual_confirmed_cases=[]
others=np.sum(country_confirmed_cases[10:])
for i in range(len(country_confirmed_cases[:10])):
    visual_unique_countries.append(unique_countries[i])
    visual_confirmed_cases.append(country_confirmed_cases[i])

    visual_unique_countries.append('others')
    visual_confirmed_cases.append(others)

In [ ]:
#visualize the 10 countries
plt.figure(figsize=(32,18))
plt.barh(visual_unique_countries,visual_confirmed_cases)
plt.title('Number of Covid-19 confirmed cases in Countries/Regions', size=20)
plt.show

In [ ]:
# using a pie chart to visualize the total number of cases in the top 10 countries
c=random.choices(list(mcolors.CSS4_COLORS.values()),k=len(unique_countries))
plt.figure(figsize=(15,15))
plt.title('Top 10 Countries Covid-19 confirmed cases')
plt.pie(visual_confirmed_cases,colors=c)
plt.legend(visual_unique_countries,loc='best')
plt.show

In [ ]:
# using a pie chart to visualize the total number of cases in the top 10 countries apart from the US
c=random.choices(list(mcolors.CSS4_COLORS.values()),k=len(unique_countries))
plt.figure(figsize=(15,15))
plt.title('Top 10 Countries Covid-19 confirmed cases')
plt.pie(visual_confirmed_cases[1:],colors=c)
plt.legend(visual_unique_countries[1:],loc='best')
plt.show

In [120]:
#BUILDING MODEL
#LINEAR REGRESSION
from sklearn.linear_model import LinearRegression
linear_model = LinearRegression(normalize=True, fit_intercept=True)
linear_model.fit(X_train_confirmed, y_train_confirmed)
test_linear_pred = linear_model.predict(X_test_confirmed)
linear_pred = linear_model.predict(future_forcast)
print('MAE:', mean_absolute_error(test_linear_pred, y_test_confirmed))
print('MSE:',mean_squared_error(test_linear_pred, y_test_confirmed))

NameError: name 'X_train_confirmed' is not defined

In [ ]:
#PREDICTIVE MODELS
# BUILDING SVM MODEL
kernel= ['poly', 'sigmoid', 'rbf']
c=[0.01,0.1,1,10]
gamma=[0.01,0.1,1]
epsilon=[0.01,0.1,1]
shrinking=[True,False]
svm_grid={'kernel':kernel,'C':c,'gamma':gamma,'epsilon':epsilon,'shrinking':shrinking}

svm=SVR()
svm_search=RandomizedSearchCV(svm,svm_grid,scoring='neg_mean_squared_error',cv=3,return_train_score=True,n_jobs=-1,n_iter=40,verbose=1)
svm_search.fit(X_train_confirmed,Y_train_confirmed)

In [ ]:
svm_search.best_params_

In [ ]:
svm_confirmed=svm_search.best_estimator_
svm_pred=svm_confirmed.predict(future_forecast)
